# Tensorflow High Level APIs introduction

This notebook introduces the scala_tensorflow library high-level APIs (Dataset and Estimators) for a simple regression model case.

Summary:

- load data into a Tensor, make a Dataset
- Create a model Estimator
- Train
- Tensorflow training vizualization
- Save Model
- Model Signature
- Infer


In [1]:
interp.load.ivy(
  coursier.Dependency(
    module = coursier.Module("org.platanios", "tensorflow_2.12"),
    version = "0.4.1",
    // replace with linux-gpu-x86_64 on linux with nvidia gpu or with darwin-cpu-x86_64 on macOS 
    attributes = coursier.Attributes("", "darwin-cpu-x86_64")
  )
)
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")


In [2]:
import sys.process._
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.learn._
import org.platanios.tensorflow.api.learn.layers._
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator
import org.platanios.tensorflow.data.image.MNISTLoader
import org.platanios.tensorflow.api.core.client.FeedMap

import java.nio.file.Paths
import scala.util.Random

import sys.process._

import org.platanios.tensorflow.api._

import org.platanios.tensorflow.api.learn._

import org.platanios.tensorflow.api.learn.layers._

import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator

import org.platanios.tensorflow.data.image.MNISTLoader

import org.platanios.tensorflow.api.core.client.FeedMap


import java.nio.file.Paths

import scala.util.Random

In [3]:
val rootDir = sys.env("HOME") + "/data/immo/"

rootDir: String = "/Users/xavier/data/immo/"

In [4]:
s"head -3 ${rootDir}immo.csv"!

Price,Surface,Land,Beds,Sidings
160,110,896,2,4
189,120,435,3,2


res3: Int = 0

In [5]:
val bufferedSource = scala.io.Source.fromFile(s"${rootDir}immo.csv")
lazy val tt = bufferedSource.getLines.drop(1).toVector
                       .map(_.split(",").map(_.trim.toFloat))
                       .map(arr => Tensor(arr(0), arr(1), arr(2), arr(3), arr(4)))

bufferedSource: scala.io.BufferedSource = <iterator>
tt: Vector[Tensor[Float]] = <lazy>

In [6]:
val sess = Session()

sess: Session = org.platanios.tensorflow.api.core.client.Session@4f36e543

In [7]:
val dataTensor = Tensor(tt:_*)

val meanD = tf.mean(dataTensor, axes = Seq(0), keepDims = true, name = "Mean")
val varianceD = tf.mean( tf.squaredDifference(meanD, tf.stopGradient(dataTensor)), axes = Seq(0), keepDims = true, name = "Variance")

val dataScaled = tf.divide(tf.subtract(dataTensor, meanD), tf.sqrt(varianceD))
val dataScaledTensor = sess.run(fetches = dataScaled)

val trainFeatures = tf.data.datasetFromTensorSlices(dataScaledTensor(---, 1::))
val trainLabels   = tf.data.datasetFromTensorSlices(dataScaledTensor(---, 0))


dataTensor: Tensor[Float] = Tensor[Float, [25, 5]]
meanD: Output[Float] = Output(Mean, 0)
varianceD: Output[Float] = Output(Variance, 0)
dataScaled: Output[Float] = Output(Divide, 0)
dataScaledTensor: Tensor[Float] = Tensor[Float, [25, 5]]
trainFeatures: ops.data.Dataset[Output[Float]] = Dataset[TensorSlicesDataset]
trainLabels: ops.data.Dataset[Output[Float]] = Dataset[TensorSlicesDataset]

In [8]:
val trainData =
  trainFeatures.zip(trainLabels)
      .repeat()
      //.shuffle(10)
      .batch(250)
      .prefetch(1000)

trainData: ops.data.Dataset[(Output[Float], Output[Float])] = Dataset[TensorSlicesDataset/Zip/Repeat/Batch/Prefetch]

In [9]:
val input = Input(FLOAT32, Shape(-1, 4))
val trainInput = Input(FLOAT32, Shape(-1))

input: Input[Output[Float]] = org.platanios.tensorflow.api.learn.layers.Input@6eba0304
trainInput: Input[Output[Float]] = org.platanios.tensorflow.api.learn.layers.Input@86b4bf4

In [10]:
import org.platanios.tensorflow.api.ops.variables._

import org.platanios.tensorflow.api.ops.variables._

In [11]:
val layers =  Linear[Float]("Layer_0", 1)
val loss = L2Loss[Float, Float]("Loss") >> ScalarSummary(name = "Loss", tag = "Loss")

layers: Linear[Float] = Linear(
  "Layer_0",
  1,
  true,
  RandomNormalInitializer(Tensor[Float, []], Tensor[Float, []], None),
  RandomNormalInitializer(Tensor[Float, []], Tensor[Float, []], None)
)
loss: Compose[(Output[Float], Output[Float]), Output[Float], Output[Float]] = Compose(
  "Loss",
  L2Loss("Loss"),
  ScalarSummary(
    "Loss",
    "Loss",
    null,
    Set(org.platanios.tensorflow.api.core.Graph$Keys$SUMMARIES$@2537a5e)
  )
)

In [12]:
val optimizer = tf.train.AdaGrad(1.0f)//tf.train.GradientDescent(1e-6f)
val model = Model.simpleSupervised(input, trainInput, layers, loss, optimizer)

optimizer: ops.training.optimizers.AdaGrad = org.platanios.tensorflow.api.ops.training.optimizers.AdaGrad@7d8a94c4
model: SupervisedTrainableModel[Output[Float], Output[Float], Output[Float], Output[Float], Float] = org.platanios.tensorflow.api.learn.Model$$anon$1@45ce7015

In [13]:
import org.platanios.tensorflow.api.learn.hooks._
import org.platanios.tensorflow.api.config.TensorBoardConfig


val summariesDir = java.nio.file.Paths.get("/tmp/001tfsc")

val estimator = InMemoryEstimator(
  modelFunction = model,
  configurationBase = Configuration(Some(summariesDir)),
  trainHooks = Set(
    SummarySaver(summariesDir, StepHookTrigger(10)),
    tf.learn.StepRateLogger(log = true, summaryDir = summariesDir, trigger = tf.learn.StepHookTrigger(100)),
    tf.learn.LossLogger(trigger = tf.learn.StepHookTrigger(10)),
    CheckpointSaver(summariesDir, StepHookTrigger(10))),
  tensorBoardConfig = TensorBoardConfig(summariesDir))

import org.platanios.tensorflow.api.learn.hooks._

import org.platanios.tensorflow.api.config.TensorBoardConfig



summariesDir: java.nio.file.Path = /tmp/001tfsc
estimator: InMemoryEstimator[Output[Float], (Output[Float], Output[Float]), Output[Float], Output[Float], Float, (Output[Float], (Output[Float], Output[Float]))] = org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator@76d79fa6

In [14]:
estimator.train(() => trainData, StopCriteria(maxSteps = Some(1000)))

2019-05-23 16:28:32.226 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 0.
2019-05-23 16:28:32.230 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-0'.
2019-05-23 16:28:32.277 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-0'.
2019-05-23 16:28:32.292 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    N/A    ) Step:      0, Loss: 139637.3438
2019-05-23 16:28:32.337 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 10.
2019-05-23 16:28:32.337 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-10'.
2019-05-23 16:28:32.355 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-10'.
2019-05-23 16:28:32.356 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.065 s) Step:     10, Loss: 31250.3281
2019-05-23 16:28:32.36

2019-05-23 16:28:32.740 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-160'.
2019-05-23 16:28:32.741 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.023 s) Step:    160, Loss: 31249.9492
2019-05-23 16:28:32.749 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 170.
2019-05-23 16:28:32.749 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-170'.
2019-05-23 16:28:32.761 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-170'.
2019-05-23 16:28:32.762 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.020 s) Step:    170, Loss: 31249.9492
2019-05-23 16:28:32.770 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 180.
2019-05-23 16:28:32.771 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-180'.
2019-05-23 16:

2019-05-23 16:28:33.072 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.022 s) Step:    320, Loss: 31249.9492
2019-05-23 16:28:33.080 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 330.
2019-05-23 16:28:33.080 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-330'.
2019-05-23 16:28:33.091 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-330'.
2019-05-23 16:28:33.092 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.020 s) Step:    330, Loss: 31249.9492
2019-05-23 16:28:33.101 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 340.
2019-05-23 16:28:33.101 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-340'.
2019-05-23 16:28:33.112 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-340'.
2019-05-23 16:

2019-05-23 16:28:33.406 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-490'.
2019-05-23 16:28:33.415 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-490'.
2019-05-23 16:28:33.416 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.016 s) Step:    490, Loss: 31249.9492
2019-05-23 16:28:33.422 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 500.
2019-05-23 16:28:33.423 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-500'.
2019-05-23 16:28:33.432 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-500'.
2019-05-23 16:28:33.432 [scala-interpreter-1] INFO  Learn / Hooks / Step Rate - Steps/Sec: 549.45
2019-05-23 16:28:33.432 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.016 s) Step:    500, Loss: 31249.9492
2019-05-23 16:28:33.439 [scala-inte

2019-05-23 16:28:33.701 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-650'.
2019-05-23 16:28:33.701 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.018 s) Step:    650, Loss: 31249.9492
2019-05-23 16:28:33.708 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 660.
2019-05-23 16:28:33.708 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-660'.
2019-05-23 16:28:33.720 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-660'.
2019-05-23 16:28:33.721 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.019 s) Step:    660, Loss: 31249.9492
2019-05-23 16:28:33.727 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 670.
2019-05-23 16:28:33.727 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-670'.
2019-05-23 16:

2019-05-23 16:28:33.977 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.016 s) Step:    810, Loss: 31249.9492
2019-05-23 16:28:33.982 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 820.
2019-05-23 16:28:33.983 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-820'.
2019-05-23 16:28:33.993 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-820'.
2019-05-23 16:28:33.994 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.016 s) Step:    820, Loss: 31249.9492
2019-05-23 16:28:33.999 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 830.
2019-05-23 16:28:34.000 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-830'.
2019-05-23 16:28:34.011 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-830'.
2019-05-23 16:

2019-05-23 16:28:34.248 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-980'.
2019-05-23 16:28:34.257 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-980'.
2019-05-23 16:28:34.258 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.016 s) Step:    980, Loss: 31249.9492
2019-05-23 16:28:34.263 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 990.
2019-05-23 16:28:34.263 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/001tfsc/model.ckpt-990'.
2019-05-23 16:28:34.272 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/001tfsc/model.ckpt-990'.
2019-05-23 16:28:34.273 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    0.015 s) Step:    990, Loss: 31249.9492
2019-05-23 16:28:34.278 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 1000.
2019-05-23 16

In [15]:
val scaledPreds = estimator.infer(() => dataScaledTensor(---, 1::))
val predsOp = tf.add(meanD(---,0),tf.multiply(scaledPreds,tf.sqrt(varianceD(---,0))))
//val preds = tf.sum(meanD(---,0), tf.multiply(scaledPreds, tf.sqrt(varianceD(---,0))))
val (preds, m, sd2) = sess.run(fetches = (predsOp,meanD,varianceD))

scaledPreds: Tensor[Float] = Tensor[Float, [25, 1]]
predsOp: Output[Float] = Output(Add, 0)
preds: Tensor[Float] = Tensor[Float, [25, 1]]
m: Tensor[Float] = Tensor[Float, [1, 5]]
sd2: Tensor[Float] = Tensor[Float, [1, 5]]

In [16]:
scaledPreds(---, 0).entriesIterator.toSeq

res15: Seq[Float] = Stream(
  4.6393488E-8F,
  4.023166E-8F,
  4.1822602E-8F,
  4.5064894E-8F,
  4.6525216E-8F,
  4.5800387E-8F,
  4.1290104E-8F,
  4.572816E-8F,
  4.533636E-8F,
  4.0941693E-8F,
  3.9265426E-8F,
  4.068332E-8F,
  4.1707455E-8F,
  4.283643E-8F,
  3.897924E-8F,
  4.6642224E-8F,
  4.2617767E-8F,
  4.3711232E-8F,
  4.6393488E-8F,
  4.0619714E-8F,
  4.5502734E-8F,
  4.2315065E-8F,
  5.1046392E-8F,
  4.7040473E-8F,
  4.126023E-8F
)

In [18]:
sd2(---,0).entriesIterator.toSeq//.map(_)

res17: Seq[Float] = Stream(2033.5618F)

In [19]:
dataScaledTensor(---, 0).entriesIterator.toSeq.map(x =>x*x).reduce(_+_)

res18: Float = 25.0F